<a href="https://colab.research.google.com/github/sungshin52/a-PyTorch-Tutorial-to-Object-Detection/blob/master/ssd_Object_Detection_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Git에서 fork 해오기**



In [ ]:
!git clone https://github.com/sungshin52/a-PyTorch-Tutorial-to-Object-Detection.git

Cloning into 'a-PyTorch-Tutorial-to-Object-Detection'...
remote: Enumerating objects: 249, done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 249
Receiving objects: 100% (249/249), 175.95 MiB | 38.42 MiB/s, done.
Resolving deltas: 100% (117/117), done.


**2. create_data_lists.py 실행하기**

create_data_lists.py의 경로를 다음과 같이 수정

voc07_path='/content/drive/MyDrive/dataset/VOC2007',
                      voc12_path='/content/drive/MyDrive/dataset/VOC2012',
                      output_folder='/content/drive/MyDrive/dataset'


In [ ]:
%cd /content/drive/MyDrive/dataset
!python /content/a-PyTorch-Tutorial-to-Object-Detection/create_data_lists.py

/content/drive/MyDrive/dataset

There are 16551 training images containing a total of 49653 objects. Files have been saved to /content/drive/MyDrive/dataset.

There are 4952 test images containing a total of 14856 objects. Files have been saved to /content/drive/MyDrive/dataset.


**3. 이미지 데이터 학습**

batch size를 16, iterations를 6000, workers를 2, decaying learning rate를 [4000,5000]으로 낮춰 Epoch 수를 11로 수정

In [ ]:
!python /content/a-PyTorch-Tutorial-to-Object-Detection/train.py

/content/drive/MyDrive/dataset

Loaded base model.

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
Epoch: [0][0/1035]	Batch Time 20.387 (20.387)	Data Time 5.344 (5.344)	Loss 23.8458 (23.8458)	
Epoch: [0][200/1035]	Batch Time 1.067 (2.523)	Data Time 0.000 (1.403)	Loss 6.9530 (10.2460)	
Epoch: [0][400/1035]	Batch Time 1.037 (2.520)	Data Time 0.000 (1.435)	Loss 5.9313 (8.4107)	
Epoch: [0][600/1035]	Batch Time 1.064 (2.513)	Data Time 0.000 (1.439)	Loss 5.9415 (7.6243)	
Epoch: [0][800/1035]	Batch Time 5.679 (2.524)	Data Time 4.639 (1.455)	Loss 5.9519 (7.1760)	
Epoch: [0][1000/1035]	Batch Time 1.039 (2.516)	Data Time 0.000 (1.450)	Loss 5.5143 (6.8767)	
Epoch: [1][0/1035]	Batch Time 2.525 (2.525)	Data Time 1.484 (1.484)	Loss 5.5953 (5.5953)	
Epoch: [1][200/1035]	Batch Time 1.047 (1.095)	Data Time 0.001 (0.009)	Loss 5.6849 (5.5168)	
Epoch: [

**4. model.py의 484~506번째 줄 에러를 다음과 같이 고치기**

                suppress = torch.zeros((n_above_min_score)).bool().to(device)  # (n_qualified)

                # Consider each box in order of decreasing scores
                for box in range(class_decoded_locs.size(0)):
                    # If this box is already marked for suppression
                    if suppress[box] == 1:
                        continue

                    # Suppress boxes whose overlaps (with this box) are greater than maximum overlap
                    # Find such boxes and update suppress indices
                    suppress = suppress | (overlap[box] > max_overlap)
                    # The max operation retains previously suppressed boxes, like an 'OR' operation

                    # Don't suppress this box, even though it has an overlap of 1 with itself
                    suppress[box] = 0

                # Store only unsuppressed boxes for this class
                image_boxes.append(class_decoded_locs[~suppress])
                image_labels.append(
                    torch.LongTensor(
                        (~suppress).sum().item() * [c]).to(device)
                )
                image_scores.append(class_scores[~suppress])

**5. detect.py의 67번째 줄 font 에러를 다음과 같이 고치기**

font = ImageFont.load_default()

**99번째 줄 img_path에 들어가는 이미지 경로를 detection 원하는 이미지 경로로 바꾼 후 실행**

**102번째 줄을 다음과 같이 고쳐 detection 수행한 이미지 저장 (min_score 올려 민감도를 낮춤)**

detect(original_image, min_score=0.4, max_overlap=0.5, top_k=200).save("./out.jpg")

**Video detection의 경우 ssd_detection_video.py 참고**

In [ ]:
%cd /content/drive/MyDrive/dataset
!python /content/a-PyTorch-Tutorial-to-Object-Detection/detect.py

/content/drive/MyDrive/dataset

Loaded checkpoint from epoch 11.

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
총 Frame 갯수: 259
더 이상 처리할 frame이 없습니다.
